# PART 1: intro

> `makemore` takes one text file as input, where each line is assumed to be one training thing, and generates more things like it. Under the hood, it is an autoregressive character-level language model, with a wide choice of models from bigrams all the way to a Transformer (exactly as seen in GPT).

## reading and exploring the dataset

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/karpathy/makemore/refs/heads/master/names.txt"
words = pd.read_csv(url, header=None).iloc[:, 0].tolist()

print(words[:10])
print(len(words))

In [ ]:
print("No of chars for the shortest word: ", min(len(w) for w in words))
print("No of chars for the longest word: ", max(len(w) for w in words))

By looking into (1) the order of characters in individual word, and (2) that pattern for the whole dataset of 32k words, we will try to infer which character is likely to follow a character or chain of characters.

We will first building a `bigrams` languague model - which only works will 2 characters at a time - look at the current character and try to predict the next one. We are just following this local structure!

It's just a simple (and weak) model but a good way to start.

## exploring the `bigrams` in the dataset

In [ ]:
for w in words[:3]:
  chs = ['<S>'] + list(w) + ['<E>'] # special start and ending token, `list()` will turn all character in word to list
  for ch1, ch2 in zip(chs, chs[1:]):
    print(ch1, ch2)

## counting `bigrams` in a python dictionary

In order to learn statistics about what character is more likely to follow another character, the simplest way is `counting`.

In [ ]:
b = {} # dict to store all pair of character
for w in words[:5]: # do it for first five words
  chs = ['<S>'] + list(w) + ['<E>']
  for ch1, ch2 in zip(chs, chs[1:]):
    bigram = (ch1, ch2)
    b[bigram] = b.get(bigram, 0) + 1
    # print(ch1, ch2)

In [ ]:
sorted(b.items(), key = lambda kv: -kv[1])

## counting `bigrams` in a 2D `torch` tensor ("training the model")

Instead of using Python dictionary, we will use `torch` 2D array to store this information.

In [ ]:
import torch

In [ ]:
a = torch.zeros((3,5), dtype=torch.int32)
a

How can we access/assign a value in torch array:

In [ ]:
a[1:3] = 10
a

Now the english alphabet contain 26 characters, we will need to capture the `<S>` and `<E>` also. So it would be 28 x 28 array.

In [ ]:
N = torch.zeros((28,28), dtype=torch.int32)

This will collect all the characters used in our dataset (join all words to a massive string and pass it to a `set()`, which will remove duplicate). With such a large dataset, all the english characters were used.

In [ ]:
chars = sorted(list(set(''.join(words))))
len(chars) # 26 

# with index
stoi = {s:i for i,s in enumerate(chars)}
stoi['<S>'] = 26
stoi['<E>'] = 27
stoi

In [ ]:
for w in words:
  chs = ['<S>'] + list(w) + ['<E>']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    N[ix1, ix2] += 1

## visualizing the `bigram` tensor

In [ ]:
itos = {i:s for s, i in stoi.items()}

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(16,16))
plt.imshow(N, cmap='Blues')
for i in range(28):
  for j in range(28):
    # plot character strings with number of time
    chstr = itos[i] + itos[j]
    plt.text(j, i, chstr, ha="center", va="bottom", color="gray")
    plt.text(j, i, N[i, j].item(), ha="center", va="top", color="gray")
plt.axis('off')

## deleting spurious (S) and (E) tokens in favor of a single `.` token

`<S>`, and `<E>` look a bit annoying. let's replace them by simple `.`.

In [ ]:
N = torch.zeros((27,27), dtype=torch.int32)
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}

for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    N[ix1, ix2] += 1

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(16,16))
plt.imshow(N, cmap='Blues')
for i in range(27):
    for j in range(27):
        chstr = itos[i] + itos[j]
        plt.text(j, i, chstr, ha="center", va="bottom", color='gray')
        plt.text(j, i, N[i, j].item(), ha="center", va="top", color='gray')
plt.axis('off')

## sampling from the model

Taking the first column of the array.

In [ ]:
N[0]

Column-wise probability.

In [ ]:
p = N[0].float()
p = p / p.sum()
p

Creating random number with Pytorch generator at a state.

In [ ]:
g = torch.Generator().manual_seed(2147483647)
p_test = torch.rand(3, generator=g)
p_test = p_test / p_test.sum()

In [ ]:
torch.multinomial(p_test, num_samples=100, replacement=True, generator=g)

Now back to our data, generate a tensor with 1 value from the `p` vector.

In [ ]:
g = torch.Generator().manual_seed(2147483647)
ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
itos[ix]

Let's automate it:

In [ ]:
g = torch.Generator().manual_seed(2147483647)

ix = 0
while True:
  p = N[ix].float()
  p = p / p.sum()
  ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
  print(itos[ix])
  if ix == 0:
    break

And more, joining the last result to single word, and make new 10 names:

In [ ]:
g = torch.Generator().manual_seed(2147483647)

for i in range(10):
  
  out = []
  ix = 0
  while True:
    p = N[ix].float()
    p = p / p.sum()

    # p = torch.ones(27) / 27.0
    # the result look terrible, but compare to an un-trained model for eg p - uncomment to code above, they are still like names.
    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print(''.join(out))

## efficiency! vectorized normalization of the rows, tensor broadcasting

We just fetching a row of `N` from the counts matrix, and then always do the same things: converting to float, dividing. That's not efficient! We now will optimize this:

In [ ]:
P = N.float()
# param 1 helps summing horizontally, by rows
# keepdim keeps the dimension the output is still 2D array with 1 column for each row, not a vertical vector entirely
# tensor already support to broadcast the row sum allowing this dividing (keepdim helped not to mess the broadcast)
P /= P.sum(1, keepdim=True)
# inplace operator instead of P = P / P.sum(1, keepdim=True), take care of memory!

In [ ]:
g = torch.Generator().manual_seed(2147483647)

for i in range(10):
  
  out = []
  ix = 0
  while True:
    p = P[ix]
    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print(''.join(out))

## loss function (the negative log likelihood of the data under our model)

We've just trained and sampled from the model, iteratively sampled the next character and fed it in each time and got the next one. Now we need to somehow measure the quality of the model.

How good is it in predicting? Gimme a number!

In [ ]:
# showing bigram for the first 3 words, along with the probability inferred by our model (`P`)
# the higher the prob, the better of prediction
# since a fair (under no data) probability of occuring a character is roughly 1/27 ~ 4%, any prob higher than 4% should be good
# we need to combine all the prob to a single 1 number, measuring how good is our model?
# since multiplying all the prob resulting a very very small number, we will approach by the log likelihood function
# the log likelihood is just the sum of log of individual multiplier

log_likelihood = 0.0
for w in words[:3]:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    prob = P[ix1, ix2]
    log_prob = torch.log(prob)
    log_likelihood += log_prob
    print(f'{ch1}{ch2}: {prob:.4f} {log_prob:.4f}')

print(f'{log_likelihood=}') # print both the variable name and its value, for the first 3 words

If all the probs equal to 1, the logs will be 0. If they close to 0, the logs will be more negative. We want to use this as a loss function, meaning lower the better, so we will invert it:

In [ ]:
neg_log_likelihood = 0.0
n = 0
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    prob = P[ix1, ix2]
    log_prob = torch.log(prob)
    neg_log_likelihood += -log_prob
    n += 1

print(f'{neg_log_likelihood=}')
print(f'{neg_log_likelihood/n}')

Finally we insert a count and calculate the "normalized" (or average) negative log likelihood. The lower of this number, the better model we have.

You can test with your name:

In [ ]:
neg_log_likelihood = 0.0
n = 0
for w in ['tuan']:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    prob = P[ix1, ix2]
    log_prob = torch.log(prob)
    neg_log_likelihood += -log_prob
    n += 1
    print(f'{ch1}{ch2}: {prob:.4f} {log_prob:.4f}')
print(f'{neg_log_likelihood=}')
print(f'{neg_log_likelihood/n}')

`tu` is not common in our dataset.

## model smoothing with fake counts

For a pair of bigram that does not exist in the dataset, for eg `jq`, the prob will be zero and log likelihood will be infinity. We can kind of smooth our model by adding constant "fake counts" to the model:

In [ ]:
#| eval: false
P = (N+1).float()

`1` is decent number, the more you added, you'll have a more uniformed distribution, the less, the more peaked distribution you have.

# PART 2: the neural network approach - intro

Now we will try to cast the problem of bigram character level of language modeling into the **neural network framework**. We first understand how to feed it in with 1 point dataset - only the first word `emma`:

## creating the `bigram` dataset for the neural net

In [ ]:
# creating training set of bigram(x, y)
xs, ys = [], []

for w in words[:1]:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    xs.append(ix1)
    ys.append(ix2)

xs = torch.tensor(xs) # both .tensor() and .Tensor() work!
ys = torch.tensor(ys)
# https://stackoverflow.com/questions/51911749/what-is-the-difference-between-torch-tensor-and-torch-tensor
# .tensor() infers dtype as int64 while .Tensor() infers dtype as float32, in this case

The input and output tensor for the first word will be look like this:

```{md}
> print(ch1,ch2)
. e
e m
m m
m a
a .
> xs
tensor([ 0,  5, 13, 13,  1])
> ys
tensor([ 5, 13, 13,  1,  0])
```

## feeding integers into neural nets? one-hot encodings

In [ ]:
import torch.nn.functional as F

xenc = F.one_hot(xs, num_classes=27).float() # remember to cast integer to float, which can be fed to neural nets
xenc

In [ ]:
xenc.shape

In [ ]:
plt.imshow(xenc, cmap="Blues")

## the "neural net": one linear layer of neurons implemented with matrix multiplication

In [ ]:
W = torch.randn((27, 1)) # fulfill a tensor with random number followed normal distribution, 1 is indicating 1 single neuron
xenc @ W # @ is matrix mult operator in PyTorch
# (5, 27) @ (27, 1) will result (5, 1) matrix
# no bias is added for now

This is 1 neuron only, now we want to evaluate all 27 characters using only 5 inputs from the first word, we'll make 27 neurons:

In [ ]:
W = torch.randn((27, 27))
xenc @ W 
# (5, 27) @ (27, 27) will result (5, 27) matrix
# no bias is added for now

## transforming neural net outputs into probabilities: the softmax

So far we have fed 5 inputs to 27 neurons for 27 characters in **the first layer** of the neural net. We notice that the output number ranges from negative to positive while we want "how likely of the next characters". It would be counts, or probs, hence we exponentiate the logits, then dividing row-wise total to get the prob of each character.

This is call the softmax!

In [ ]:
logits = xenc @ W # log-counts
counts = logits.exp() # equivalent N
probs = counts / counts.sum(1, keepdims=True)
probs

## summary, preview to next steps, reference to `micrograd`

In [ ]:
#| eval: false
# randomly initialize 27 neurons' weights. each neuron receives 27 inputs
g = torch.Generator().manual_seed(2147483647) # to make sure we all have same random
W = torch.randn((27, 27), generator=g)
xenc = F.one_hot(xs, num_classes=27).float() # input to the network: one-hot encoding
logits = xenc @ W # predict log-counts
counts = logits.exp() # counts, equivalent to N
probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
# btw: the last 2 lines here are together called a 'softmax'

In [ ]:
probs.shape

Below is detail explaination for each example from our 5-datapoint dataset.

In [ ]:
nlls = torch.zeros(5)
for i in range(5):
  # i-th bigram:
  x = xs[i].item() # input character index
  y = ys[i].item() # label character index
  print('--------')
  print(f'bigram example {i+1}: {itos[x]}{itos[y]} (indexes {x},{y})')
  print('input to the neural net:', x)
  print('output probabilities from the neural net:', probs[i])
  print('label (actual next character):', y)
  p = probs[i, y]
  print('probability assigned by the net to the the correct character:', p.item())
  logp = torch.log(p)
  print('log likelihood:', logp.item())
  nll = -logp
  print('negative log likelihood:', nll.item())
  nlls[i] = nll

print('=========')
print('average negative log likelihood, i.e. loss =', nlls.mean().item())

## vectorized loss

In [ ]:
loss = - probs[torch.arange(5), ys].log().mean()
loss

## backward and update, in PyTorch

In [ ]:
#| eval: false
# backward pass
W.grad = None # set to zero the gradient
loss.backward()
# this can not yet run for now, PyTorch require the specification of require_grad

In [ ]:
#| eval: false
W.data += -0.1 * W.grad

## putting everything together

In [ ]:
# create the dataset
xs, ys = [], []
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1, ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    xs.append(ix1)
    ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('number of examples: ', num)

# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True)

In [ ]:
# gradient descent
for k in range(1): # after run 100 times, shorten the notebook
  
  # forward pass
  xenc = F.one_hot(xs, num_classes=27).float() # input to the network: one-hot encoding
  logits = xenc @ W # predict log-counts
  counts = logits.exp() # counts, equivalent to N
  probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
  loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean() # regularization loss
  print(loss.item())
  
  # backward pass
  W.grad = None # set to zero the gradient
  loss.backward()
  
  # update
  W.data += -50 * W.grad

Looking back to the backprogation in the [lesson 1](https://lktuan.github.io/blog/2024-06-17-nn-z2h-p1/#doing-gradient-descent-optimization-manually-training-the-network), everything look similar here:

| Part          | This neural nets for bigram language modeling                                | Neural nets introduced in the lesson 1                                                        |
|------------------|----------------------------|--------------------------|
| Forward pass  | `probs` , use negative log likelihood as loss function, doing classification | `ypred` , use MSE for loss function, doing regression                                         |
| Backward pass | Same, offered by Torch.                                                      | Set grad of params to be zeros and do backpropagation.                                        |
| Update loss   | Same                                                                         | Update the parameters, change the parameters following opposite direction to reduce the loss. |

## note 1: one-hot encoding really just selects a row of the next Linear layer's weight matrix

Look at the below code, `xenc @ W` is (5, 27) \@ (27, 27) that will result (5, 27) matrix. Each `ix` row of that 5-rows result matrix should be the selection of corresponding character rows in the `W`.

In [ ]:
#| eval: false
logits = xenc @ W # predict log-counts
counts = logits.exp() # counts, equivalent to N
probs = counts / counts.sum(1, keepdims=True) # probabilities for next character

So in this gradient-based framework, we start with a random array of parameters. By optimizing the loss function we will get the same result with the bigram approach (`W` and `N` are almost the same, it's log count here why is count in bigram). That's why we obtained the same loss!

The neural networks offer more flexibility!

## note 2: model smoothing as regularization loss

Same with smoothing technique when we've doing the bigram model, gradient-based framework have an equivalent way for smoothing. We will try to incentivize `W` to be near zero. We augment to loss function by adding this: `0.01*(W**2).mean()`.

In [ ]:
#| eval: false
loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()

## sampling from the neural net

In [ ]:
# finally, sample from the 'neural net' model
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
  
  out = []
  ix = 0
  while True:
    
    # ----------
    # BEFORE:
    # p = P[ix]
    # ----------
    # NOW:
    xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    p = counts / counts.sum(1, keepdims=True) # probabilities for next character
    # ----------
    
    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print(''.join(out))

## conclusion

What we've gone through:

-   introduced bigrams language model? how we can train, sample, and evaluate the model;
-   we modeled by 2 different ways:
    -   1st: counted out the freq of bigram and normalized it;
    -   2nd: used a negative log likelihood loss as a guide to optimizing the counts matrix/array in a gradient-based framework;
    -   we obtained the same result!
-   gradient-based framework is more flexible. We've just modeled the simplest/dumpiest language model. In next lessons, we will complexify it.

We are on the way out to **transformer**!

Thank you, Andrej!

# resources

1.  YouTube video lecture: <https://www.youtube.com/watch?v=PaCmpygFfXo>
2.  Jupyter notebook files: <https://github.com/karpathy/nn-zero-to-hero/blob/master/lectures/makemore/makemore_part1_bigrams.ipynb>
3.  `makemore` Github repo: <https://github.com/karpathy/makemore>